In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.nn.modules.utils import _pair
from torch.nn import init

import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import sys
import math
import numpy as np
import time

import utils as utils
# import deepshift.kernels
import ste as ste

log2 = math.log(2)

In [72]:
class LinearShift(nn.Module):
    def __init__(self, in_features, 
                 out_features, 
                 bias=True, 
#                  check_grad=False, 
                 freeze_sign=False, 
#                  use_kernel=False, 
#                  use_cuda=True, 
                 rounding='deterministic', 
                 weight_bits=5
                ):
        
        super(LinearShift, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
#         self.use_kernel = use_kernel
#         self.check_grad = check_grad
#         self.use_cuda = use_cuda
#         self.conc_weight = None
        self.rounding = rounding
        self.shift_range = (-1 * (2**(weight_bits - 1) - 2), 0) # we use ternary weights to represent sign
        
        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        
#         if check_grad:
#             tensor_constructor = torch.DoubleTensor # double precision required to check grad
#         else:
#             tensor_constructor = torch.Tensor # In PyTorch torch.Tensor is alias torch.FloatTensor
        
        tensor_constructor = torch.Tensor # In PyTorch torch.Tensor is alias torch.FloatTensor
        self.shift = nn.Parameter(tensor_constructor(out_features, in_features))
        self.sign = nn.Parameter(tensor_constructor(out_features, in_features), requires_grad = (freeze_sign == False))

        if bias:
            self.bias = nn.Parameter(tensor_constructor(out_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        self.shift.data.uniform_(*self.shift_range) 
        self.sign.data.uniform_(-1, 1) 
        
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.shift)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        self.shift.data = ste.clamp(self.shift.data, *self.shift_range)
        shift_rounded = ste.round(self.shift, rounding=self.rounding)
        sign_rounded_signed = ste.sign(ste.round(self.sign, rounding=self.rounding))
        weight_ps = ste.unsym_grad_mul(2**shift_rounded, sign_rounded_signed)
        print(weight_ps)
        return torch.nn.functional.linear(input, weight_ps, self.bias)


In [70]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
#         self.fc1 = nn.Linear(9216, 128)
#         self.fc2 = nn.Linear(128, 10)
        self.fc1 = LinearShift(9216, 128)
        self.fc2 = LinearShift(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        readout = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item())
        sys.stdout.write(readout)
        sys.stdout.flush()
        sys.stdout.write('\r')
        
        

In [40]:
# device = torch.device("cuda" if use_cuda else "cpu")

# kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# torch.manual_seed(args.seed)
use_cuda = False

device = torch.device("cuda" if use_cuda else "cpu")

n_epochs=1
lr=1.0
gamma=0.7
batch_size = 64
test_batch_size = 1000

kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


In [73]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)
    scheduler.step()

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [-1.2207e-04, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [ 0.0000e+00, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [ 0.0000e+00, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [ 0.0000e+00, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [ 0.0000e+00, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

...

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [-1.2207e-04, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [-1.2207e-04, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [-1.2207e-04, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

tensor([[ 9.7656e-04,  0.0000e+00,  2.4414e-04,  ...,  1.2207e-04,
          0.0000e+00, -1.2207e-04],
        [ 0.0000e+00,  1.2500e-01,  0.0000e+00,  ..., -4.8828e-04,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.8125e-03,  ...,  1.2500e-01,
          0.0000e+00, -1.9531e-03],
        ...,
        [-6.2500e-02, -2.5000e-01,  0.0000e+00,  ..., -9.7656e-04,
          1.5625e-02,  0.0000e+00],
        [-1.2207e-04, -5.0000e-01,  0.0000e+00,  ...,  7.8125e-03,
         -1.5625e-02,  1.2500e-01],
        [-7.8125e-03,  0.0000e+00,  1.9531e-03,  ..., -3.1250e-02,
         -2.4414e-04,  0.0000e+00]],
       grad_fn=<UnsymmetricGradMulFunctionBackward>)
tensor([[ 0.0000e+00, -2.5000e-01, -1.2500e-01,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03],
        [ 3.9062e-03,  9.7656e-04,  0.0000e+00,  ..., -3.1250e-02,
         -7.8125e-03,  0.0000e+00],
        [ 3.9062e-03, -3.1250e-02, -1.2207e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

KeyboardInterrupt: 